In [1]:
import pandas as pd
import os
from datetime import datetime
from numpy import nan

# Broker files

In [2]:
notebook_path = os.path.abspath('ImportBrokerFiles.ipynb')
file_ibkr = os.path.join(os.path.dirname(notebook_path), 'data', 'ibkr.csv')
file_xtb  = os.path.join(os.path.dirname(notebook_path), 'data', 'xtb.csv')

# Data normalization

## InteractiveBrokers

```json
{
  "HEADER": "DATA",
  "TRNT": "TRNT",
  "ClientAccountID": "U12046212",
  "AccountAlias": "",
  "Model": "",
  "CurrencyPrimary": "EUR",
  "FXRateToBase": "1.088",
  "AssetClass": "STK",
  "SubCategory": "COMMON",
  "Symbol": "DBKd",
  "Description": "DEUTSCHE BANK AG-REGISTERED",
  "Conid": "14121",
  "SecurityID": "DE0005140008",
  "SecurityIDType": "ISIN",
  "CUSIP": "",
  "ISIN": "DE0005140008",
  "FIGI": "BBG000BBZTH2",
  "ListingExchange": "IBIS",
  "UnderlyingConid": "",
  "UnderlyingSymbol": "",
  "UnderlyingSecurityID": "",
  "UnderlyingListingExchange": "",
  "Issuer": "",
  "IssuerCountryCode": "",
  "TradeID": "596641620",
  "Multiplier": "1",
  "RelatedTradeID": "",
  "Strike": "",
  "ReportDate": "20230829",
  "Expiry": "",
  "DateTime": "20230829;074315",
  "Put/Call": "",
  "TradeDate": "20230829",
  "PrincipalAdjustFactor": "",
  "SettleDateTarget": "20230831",
  "TransactionType": "ExchTrade",
  "Exchange": "EUIBSIL",
  "Quantity": "60",
  "TradePrice": "10.24",
  "TradeMoney": "614.4",
  "Proceeds": "-614.4",
  "Taxes": "0",
  "IBCommission": "-3",
  "IBCommissionCurrency": "EUR",
  "NetCash": "-617.4",
  "ClosePrice": "10.27",
  "Open/CloseIndicator": "O",
  "Notes/Codes": "IA",
  "CostBasis": "617.4",
  "FifoPnlRealized": "0",
  "MtmPnl": "1.8",
  "OrigTradePrice": "0",
  "OrigTradeDate": "",
  "OrigTradeID": "",
  "OrigOrderID": "0",
  "OrigTransactionID": "0", 
  "Buy/Sell": "BUY",
  "ClearingFirmID": "",
  "IBOrderID": "506972751",
  "TransactionID": "2101061418",
  "IBExecID": "00020f63.64ed8f11.01.01",
  "RelatedTransactionID": "",
  "BrokerageOrderID": "0117f5e9.0002e17f.64ed75a7.0001",
  "OrderReference": "",
  "VolatilityOrderLink": "",
  "ExchOrderID": "N/A",
  "ExtExecID": "115204175B",
  "OrderTime": "20230829;074315",
  "OpenDateTime": "",
  "HoldingPeriodDateTime": "",
  "WhenRealized": "",
  "WhenReopened": "",
  "LevelOfDetail": "EXECUTION",
  "ChangeInPrice": "0",
  "ChangeInQuantity": "0",
  "OrderType": "LMT",
  "TraderID": "",
  "IsAPIOrder": "N",
  "AccruedInterest": "0",
  "SerialNumber": "",
  "DeliveryType": "",
  "CommodityType": "",
  "Fineness": "0.0",
  "Weight": "0.0"
}
```

In [3]:
custom_date_parser      = lambda x: datetime.strptime(x, "%Y%m%d")
custom_datetime_parser  = lambda x: datetime.strptime(x, "%Y%m%d;%H%M%S")
parse_dates             = ['ReportDate', 'TradeDate', 'SettleDateTarget']
df_ibkr = pd.read_csv(file_ibkr, parse_dates=parse_dates, date_parser=custom_date_parser)

columns = ['CurrencyPrimary','FXRateToBase','AssetClass','Symbol','Description','ReportDate','DateTime','TradeDate','SettleDateTarget',
           'Quantity','TradePrice','TradeMoney','Proceeds','Taxes','IBCommission','IBCommissionCurrency','NetCash',
           'ClosePrice','CostBasis','Buy/Sell','OrderTime','OrderType','TransactionID']

df_ibkr["DateTime"] = df_ibkr["DateTime"].apply(custom_datetime_parser)
df_ibkr["OrderTime"] = df_ibkr["OrderTime"].apply(custom_datetime_parser)
df_ibkr = df_ibkr[columns]

# string: 'CurrencyPrimary','AssetClass','Symbol','Description','Buy/Sell','OrderType','IBCommissionCurrency'
# date: 'ReportDate','SettleDateTarget','TradeDate',
# datetime: 'DateTime','OrderTime',
# number:
cols_to_number = ['FXRateToBase','Quantity','TradePrice','TradeMoney','Proceeds','Taxes','IBCommission','NetCash','ClosePrice','CostBasis']
df_ibkr[cols_to_number] = df_ibkr[cols_to_number].apply(pd.to_numeric)

df_ibkr.iloc[0]

CurrencyPrimary                                 EUR
FXRateToBase                                  1.088
AssetClass                                      STK
Symbol                                         DBKd
Description             DEUTSCHE BANK AG-REGISTERED
ReportDate                      2023-08-29 00:00:00
DateTime                        2023-08-29 07:43:15
TradeDate                       2023-08-29 00:00:00
SettleDateTarget                2023-08-31 00:00:00
Quantity                                       60.0
TradePrice                                    10.24
TradeMoney                                    614.4
Proceeds                                     -614.4
Taxes                                             0
IBCommission                                   -3.0
IBCommissionCurrency                            EUR
NetCash                                      -617.4
ClosePrice                                    10.27
CostBasis                                     617.4
Buy/Sell    

### IKBR normalization for transactions table
#### columns
symbol,dt,comment,quote,shares,fee,taxes,currency,shares_left,category

In [4]:
# cols                                  = ['symbol','dt',       'comment',    'quote',      'shares',   'fee',          'taxes','currency',       'shares_left','category','amount']
df_ibkr_cols                            = ['Symbol','OrderTime','Description','TradePrice', 'Quantity', 'IBCommission', 'Taxes','CurrencyPrimary','Buy/Sell','TradeMoney','TransactionID','FXRateToBase']
df_ibkr_transactions                    = df_ibkr[df_ibkr_cols]
df_ibkr_transactions['Buy/Sell']        = df_ibkr_transactions['Buy/Sell'].apply(lambda x: x.lower())
df_ibkr_transactions['CurrencyPrimary'] = df_ibkr_transactions['CurrencyPrimary'].apply(lambda x: x.lower())
df_ibkr_transactions.rename(columns={
  'TransactionID': 'id',
  'Symbol': 'symbol',
  'OrderTime': 'dt',
  'Description': 'comment',
  'TradePrice': 'quote', 
  'Quantity': 'shares',
  'IBCommission': 'fee',
  'Taxes': 'taxes',
  'CurrencyPrimary': 'currency',
  'Buy/Sell': 'category',
  'TradeMoney': 'amount',
  'FXRateToBase': 'exchange_rate'
}, inplace=True)
df_ibkr_transactions['broker'] = 'ibkr'
df_ibkr_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   symbol         101 non-null    object        
 1   dt             101 non-null    datetime64[ns]
 2   comment        101 non-null    object        
 3   quote          101 non-null    float64       
 4   shares         101 non-null    float64       
 5   fee            101 non-null    float64       
 6   taxes          101 non-null    int64         
 7   currency       101 non-null    object        
 8   category       101 non-null    object        
 9   amount         101 non-null    float64       
 10  id             101 non-null    int64         
 11  exchange_rate  101 non-null    float64       
 12  broker         101 non-null    object        
dtypes: datetime64[ns](1), float64(5), int64(2), object(5)
memory usage: 10.4+ KB


/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_1944/849476559.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ibkr_transactions['Buy/Sell']        = df_ibkr_transactions['Buy/Sell'].apply(lambda x: x.lower())
/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_1944/849476559.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ibkr_transactions['CurrencyPrimary'] = df_ibkr_transactions['CurrencyPrimary'].apply(lambda x: x.lower())
/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/

## XTB
```json
{
  "ID": "479502259",
  "Type": "Stocks/ETF purchase",
  "Time": "27.12.2023 15:42:57",
  "Symbol": "PYPL.US",
  "Comment": "OPEN BUY 2 @ 62.30",
  "Amount": "-124.6"
}
```

In [5]:
def parse_comment(comment, field):
  category  = None
  quote     = 0
  shares    = 0
  if ' @ ' in comment:                            # OPEN BUY 2 @ 62.30
    quote = float(comment.split(' @ ')[-1])
    category = 'buy' if comment.split(' @ ')[0].startswith('OPEN') else 'sell'
    if comment.startswith('OPEN BUY'):
      left_section = comment .split(' @ ')[0]
      shares_str = left_section.split('OPEN BUY ')[-1]
      shares = float(shares_str.split('/')[0] if '/' in shares_str else shares_str)
    if comment.startswith('CLOSE BUY'):
      left_section = comment .split(' @ ')[0]
      shares_str = left_section.split('CLOSE BUY ')[-1]
      shares = float(shares_str.split('/')[-1] if '/' in shares_str else shares_str)

  if 'WHT' in comment:                            # CI.US USD WHT 30%
    category = 'taxes'
  if comment.startswith('Sec Fee adj'):           # Sec Fee adj AMZN.US 20231208
    category = 'taxes'
  if 'Free-funds Interest Tax' in comment:        # Free-funds Interest Tax 2023-11
    category = 'taxes'
  elif 'Free-funds Interest' in comment:          # Free-funds Interest 2023-11	
    category = 'taxes'
  if comment.startswith('Deposit'):               # Deposit
    category = 'deposit'
  elif 'SHR' in comment:                          # CI.US USD 1.2300/ SHR
    category = 'dividend'
  if comment.startswith('Rollover of position'):  # Rollover of position #991498946
    category = 'fee'
  if comment.startswith('Swap of position'):      # Swap of position #991498946
    category = 'fee'
  if comment.startswith('Profit of position'):    # Profit of position #991498946
    category = 'profit'
  
  if field == 'category':
    return category
  if field == 'quote':
    return quote
  if field == 'shares':
    return shares

def parse_symbol(symbol):
  return None if symbol is nan else symbol.split('.')[0]

custom_datetime_parser  = lambda x: datetime.strptime(x, "%d.%m.%Y %H:%M:%S")
parse_dates             = ['Time']
df_xtb                  = pd.read_csv(file_xtb, parse_dates=parse_dates, date_parser=custom_datetime_parser,delimiter=';')
columns                 = ['ID','Type','Time','Symbol','Comment','Amount']
df_xtb                  = df_xtb[columns]

# string: 'Type','Symbol','Comment'
# datetime: 'Time',
# number:
cols_to_number            = ['Amount']
df_xtb[cols_to_number]    = df_xtb[cols_to_number].apply(pd.to_numeric)
df_xtb['category']        = df_xtb.apply(lambda x: parse_comment(x['Comment'], field='category'), axis=1)
df_xtb['quote']           = df_xtb.apply(lambda x: parse_comment(x['Comment'], field='quote'), axis=1)
df_xtb['shares']          = df_xtb.apply(lambda x: parse_comment(x['Comment'], field='shares'), axis=1)
df_xtb['symbol']          = df_xtb['Symbol'].apply(parse_symbol)
df_xtb['currency']        = 'usd'
df_xtb['fee']             = 0
df_xtb['taxes']           = 0
df_xtb['exchange_rate']   = 1
df_xtb

,ID,Type,Time,Symbol,Comment,Amount,category,quote,shares,symbol,currency,fee,taxes,exchange_rate
0,519331363,Withholding tax,2024-03-21 11:00:17,CI.US,CI.US USD WHT 30%,-0.84,taxes,0.000,0.0000,CI,usd,0,0,1
1,519331362,Dividend,2024-03-21 11:00:17,CI.US,CI.US USD 1.4000/ SHR,2.80,dividend,0.000,0.0000,CI,usd,0,0,1
2,515765243,Stocks/ETF purchase,2024-03-14 14:11:19,GCT.US,OPEN BUY 15 @ 36.10,-541.50,buy,36.100,15.0000,GCT,usd,0,0,1
3,510894963,Stocks/ETF purchase,2024-03-05 14:44:58,LQQ.FR,OPEN BUY 0.477 @ 961.60,-500.01,buy,961.600,0.4770,LQQ,usd,0,0,1
4,510884871,Subaccount transfer,2024-03-05 14:40:32,NaN,Transfer from 2776908 to 50326960,-2000.00,None,0.000,0.0000,None,usd,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,500650782,Stocks/ETF purchase,2024-02-13 08:04:31,SXR8.DE,OPEN BUY 0.7901/2.7901 @ 489.22,-418.51,buy,489.220,0.7901,SXR8,usd,0,0,1
97,500648256,Stocks/ETF purchase,2024-02-13 08:04:17,IS0L.DE,OPEN BUY 0.7299/3.7299 @ 122.5050,-96.81,buy,122.505,0.7299,IS0L,usd,0,0,1
98,500647980,Stocks/ETF purchase,2024-02-13 08:04:15,IS3N.DE,OPEN BUY 0.5319/15.5319 @ 29.339,-16.90,buy,29.339,0.5319,IS3N,usd,0,0,1
99,500641363,Stocks/ETF purchase,2024-02-13 08:00:45,CNDX.UK,OPEN BUY 0.4819 @ 1018.94,-491.03,buy,1018.940,0.4819,CNDX,usd,0,0,1


### XTB normalization for transactions table
#### columns
symbol,dt,comment,quote,shares,fee,taxes,currency,shares_left,category,amount

In [6]:
# cols              = ['id','symbol','dt',  'comment','quote', 'shares', 'fee', 'taxes','currency','shares_left','category','amount']
df_xtb_cols         = ['ID','symbol','Time','Comment','quote', 'shares', 'fee', 'taxes','currency','category','Amount','exchange_rate']
df_xtb_transactions = df_xtb[df_xtb_cols]
df_xtb_transactions.rename(columns={
  'ID': 'id',
  'Time': 'dt',
  'Comment': 'comment',
  'Quantity': 'shares',
  'Amount': 'amount'
}, inplace=True)
df_xtb_transactions['broker'] = 'xtb'
df_xtb_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             101 non-null    int64         
 1   symbol         70 non-null     object        
 2   dt             101 non-null    datetime64[ns]
 3   comment        101 non-null    object        
 4   quote          101 non-null    float64       
 5   shares         101 non-null    float64       
 6   fee            101 non-null    int64         
 7   taxes          101 non-null    int64         
 8   currency       101 non-null    object        
 9   category       84 non-null     object        
 10  amount         101 non-null    float64       
 11  exchange_rate  101 non-null    int64         
 12  broker         101 non-null    object        
dtypes: datetime64[ns](1), float64(3), int64(4), object(5)
memory usage: 10.4+ KB


/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_1944/1021751014.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xtb_transactions.rename(columns={
/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_1944/1021751014.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xtb_transactions['broker'] = 'xtb'


## Securities

In [7]:
df1 = df_xtb_transactions
df2 = df_ibkr_transactions
df = pd.concat([df1,df2])
df.drop(columns=['id'], inplace=True)
df

,symbol,dt,comment,quote,shares,fee,taxes,currency,category,amount,exchange_rate,broker
0,CI,2024-03-21 11:00:17,CI.US USD WHT 30%,0.00000,0.000,0.0,0,usd,taxes,-0.840,1.0,xtb
1,CI,2024-03-21 11:00:17,CI.US USD 1.4000/ SHR,0.00000,0.000,0.0,0,usd,dividend,2.800,1.0,xtb
2,GCT,2024-03-14 14:11:19,OPEN BUY 15 @ 36.10,36.10000,15.000,0.0,0,usd,buy,-541.500,1.0,xtb
3,LQQ,2024-03-05 14:44:58,OPEN BUY 0.477 @ 961.60,961.60000,0.477,0.0,0,usd,buy,-500.010,1.0,xtb
4,None,2024-03-05 14:40:32,Transfer from 2776908 to 50326960,0.00000,0.000,0.0,0,usd,None,-2000.000,1.0,xtb
...,...,...,...,...,...,...,...,...,...,...,...,...
96,EUR.USD,2024-02-26 09:41:55,EUR.USD,1.08515,-500.000,-2.0,0,usd,sell,-542.575,1.0,ibkr
97,EUR.USD,2024-02-27 04:34:05,EUR.USD,1.08530,-1500.000,-2.0,0,usd,sell,-1627.950,1.0,ibkr
98,EUR.USD,2024-03-05 09:33:33,EUR.USD,1.08470,-1500.000,-2.0,0,usd,sell,-1627.050,1.0,ibkr
99,EUR.USD,2024-03-13 09:56:51,EUR.USD,1.09360,-1000.000,-2.0,0,usd,sell,-1093.600,1.0,ibkr


In [8]:
def symbol_category(symbol):
  security_category_default = 'stock'
  security_categories = {'US500': 'index', 'SXR8': 'etf', 'CNDX': 'etf', 'IS0L': 'etf', 'IS3N': 'etf'}
  return security_category_default if symbol not in security_categories.keys() else security_categories[symbol]

def symbol_exchange(symbol):
  security_exchange_default = 'NASDAQ'
  security_exchanges  = {'ALB': 'NYSE', 'BABA': 'NYSE', 'CI': 'NYSE', 'CLS': 'NYSE', 'CVNA': 'NYSE', 'DBKd':'XETR', 'HEI': 'NYSE', 'JPM': 'NYSE', 'LQQ': 'EURONEXT', 'KO': 'NYSE', 'PAYC': 'NYSE', 'T': 'NYSE'}
  return security_exchange_default if symbol not in security_exchanges.keys() else security_exchanges[symbol]

df_assets             = df.drop_duplicates(subset=['symbol'])[['symbol']]
df_assets['category'] = df_assets['symbol'].apply(symbol_category)
df_assets['exchange'] = df_assets['symbol'].apply(symbol_exchange)
df_assets['name']     = df_assets['symbol']
df_assets.query('symbol == symbol', inplace=True)
df_assets.set_index('symbol', inplace=True)
df_assets.sort_values(by=['symbol'])
df_assets.reset_index(inplace=True)
df_assets

,symbol,category,exchange,name
0,CI,stock,NYSE,CI
1,GCT,stock,NASDAQ,GCT
2,LQQ,stock,EURONEXT,LQQ
3,AAPL,stock,NASDAQ,AAPL
4,CSCO,stock,NASDAQ,CSCO
5,PYPL,stock,NASDAQ,PYPL
6,AMZN,stock,NASDAQ,AMZN
7,INMD,stock,NASDAQ,INMD
8,CVNA,stock,NYSE,CVNA
9,US500,index,NASDAQ,US500


### Import securities

In [9]:
from helpers_db import get_engine, truncate_table, insert_on_conflict_nothing_securities

engine = get_engine()
with engine.begin() as conn:
  truncate_table(conn, 'journal')
df_assets.to_sql('assets', engine, if_exists='append', index=False, method=insert_on_conflict_nothing_securities)

39

## Import Transactions

In [10]:
# df.reset_index(inplace=True)
# 
# df.set_index(['symbol', 'dt'], inplace=True)
# df.query('dt==dt')
df['shares_left'] = df['shares'].abs()
df['shares'] = df['shares'].abs()
df['amount'] = df['amount'].abs()
df['fee'] = df['fee'].abs()
df['quote'] = df['quote'].abs()
df.query("symbol=='ADP'").sort_values(by=['dt'])



,symbol,dt,comment,quote,shares,fee,taxes,currency,category,amount,exchange_rate,broker,shares_left
13,ADP,2023-07-27 10:54:12,AUTOMATIC DATA PROCESSING,252.88,3.0,1.00000,0,usd,buy,758.64,1.0,ibkr,3.0
14,ADP,2024-02-15 13:24:31,AUTOMATIC DATA PROCESSING,252.58,3.0,1.00656,0,usd,sell,757.74,1.0,ibkr,3.0


In [11]:
from helpers_db import get_engine, truncate_table, insert_on_conflict_nothing_transactions

engine = get_engine()
with engine.begin() as conn:
  truncate_table(conn, 'journal')
  truncate_table(conn, 'positions')
  truncate_table(conn, 'transactions')

df.sort_values(by=['dt']).to_sql('transactions', engine, if_exists='append', index=False, method=insert_on_conflict_nothing_transactions)


202

## Patch DBKd to DBK

In [13]:
from helpers_db import run_sql

with engine.begin() as conn:
  for table in ['assets', 'transactions', 'positions']:
    run_sql(conn, f"""UPDATE {table} SET symbol = 'DBK' WHERE symbol = 'DBKd'""")